In [ ]:
# add libraries
import requests
import json
import pandas as pd
import os
from pprint import pprint
import time

# Data Acquisition

First have to get PUUID (unique id for players) that is required to search through their matches.

To get PUUID, use LEAGUE-V4 to get a list of players to search up using SUMMONER-V4 to get PUUID. After acquiring PUUID for players, use that PUUID to query in MATCH-V5 to get recent matches. Using those matches, get the timelines for each match at 10 minute mark.

In [ ]:
# acquire data from different API endpoints
# League-V4

queue = 'RANKED_SOLO_5x5'
tier = 'DIAMOND'
division = 'I'

# pages of players
# iterate through pages 1-5 to get ~1000 players to use as sample data for now
params = {
    "page": 1
}

url = 'https://na1.api.riotgames.com/lol/'
league_url = f'{url}league/v4/entries/{queue}/{tier}/{division}'

headers = {}
headers["X-Riot-Token"] = ''

In [ ]:
league_data = []
for i in range(5):
    params["page"] = i + 1
    res = requests.get(url = league_url, params = params, headers = headers)
    league_data.append(res.json())

In [ ]:
# get the summonerId's to search up
summoner_ids = []
for page in league_data:
    for entry in page:
        summoner_ids.append(entry.get('summonerId'))

In [ ]:
len(summoner_ids)

In [ ]:
# get the PUUID and save them to a pd dataframe
# work with the first 100 puuids for now
# SUMMONER-V4

summoner_data = []

for id in summoner_ids[0:100]:
    summoner_url = f'{url}summoner/v4/summoners/{id}'
    res = requests.get(url = summoner_url, headers = headers)
    summoner_data.append(res.json())
    time.sleep(1)

In [ ]:
puuids = []
for summoner in summoner_data:
    puuids.append(summoner.get('puuid'))

In [ ]:
len(puuids)

In [ ]:
puuids[0]

In [ ]:
# get the 10 most recent games for puuid
# store into set so no duplicate games are used
last_10_games = []

params = {
    "queue": 420,
    "start": -1,
    "count": 10
}

for puuid in puuids[0:5]:
    last_10_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids"
    res = requests.get(url = last_10_url, params = params, headers = headers)
    for match_id in res.json():
        last_10_games.append(match_id)
    time.sleep(1)

In [ ]:
len(last_10_games)

In [ ]:
# get info from each match at 10 minute mark
match_id = last_10_games[0]

match_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}/timeline"
res = requests.get(url = match_url, headers = headers)

In [ ]:
match = res.json()

# Feature Engineering

participants 1-5 are blue side

6-10 are red side

WARDS:

WARD_KILL

WARD_PLACED

ITEM_DESTROYED: 2055 (control ward)

KILLS:

CHAMIPON_KILLS

OBJECTIVES:

TURRET_PLATE_DESTROYED

BUILDING_KILL

ELITE_MONSTER_KILL

Each frame is 60 seconds of gameplay.
So for each frame need to record the events by parsing through the request
like wards placed
count total kills from each frame as well

In [ ]:
print(len(match['info']['frames']))

In [ ]:
pprint(match['info']['frames'])

In [ ]:
# make a dataframe with matchid as index and features as columns